In [1]:
import os
import site
import sys

!echo "installing required python libraries, please wait..."
!{sys.executable} -m pip install --upgrade predictionguard #> /dev/null # for accessing LLM APIs
!{sys.executable} -m pip install --upgrade  "transformers>=4.38.*" #> /dev/null
!{sys.executable} -m pip install --upgrade  "datasets>=2.18.*" #> /dev/null
!{sys.executable} -m pip install --upgrade "accelerate>=0.28.*" #> /dev/null
!{sys.executable} -m pip install --upgrade faiss-cpu #> /dev/null  # for indexing
!{sys.executable} -m pip install --upgrade sentence_transformers #> /dev/null # for generating embeddings
!echo "installation complete..."

# add the location where we installed these libraries to the python pkg path (~/.local/lib/python3.9/*)
# Get the site-packages directory
site_packages_dir = site.getsitepackages()[0]

# add the site pkg directory where these pkgs are insalled to the top of sys.path
if not os.access(site_packages_dir, os.W_OK):
    user_site_packages_dir = site.getusersitepackages()
    if user_site_packages_dir in sys.path:
        sys.path.remove(user_site_packages_dir)
    sys.path.insert(0, user_site_packages_dir)
else:
    if site_packages_dir in sys.path:
        sys.path.remove(site_packages_dir)
    sys.path.insert(0, site_packages_dir)


# adding ~/.local/bin to PATH as well
home_dir = os.path.expanduser('~')
bin_path = os.path.join(home_dir, '.local', 'bin')
os.environ['PATH'] += os.pathsep + bin_path

"installing required python libraries, please wait..."


The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.


"installation complete..."


In [3]:
import os
import json

# FREE access token for usage at: tinyurl.com/pg-intel-hack
import predictionguard as pg
from getpass import getpass

In [4]:
# pg_access_token = getpass('q1VuOjnffJ3NO2oFN8Q9m8vghYc84ld13jaqdF7E')
# os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

In [5]:
import transformers

In [6]:
model_name = 'Intel/neural-chat-7b-v3-1'
model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# def generate_response(system_input, user_input):

#     # Format the input using the provided template
#     prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"

#     # Tokenize and encode the prompt
#     inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

#     # Generate a response
#     outputs = model.generate(inputs, max_length=1000, num_return_sequences=1)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     # Extract only the assistant's response
#     return response.split("### Assistant:\n")[-1]


# # Example usage
# system_input = "You are a math expert assistant. Your mission is to help users understand and solve various math problems. You should provide step-by-step solutions, explain reasonings and give the correct answer."
# user_input = "calculate 100 + 520 + 60"
# response = generate_response(system_input, user_input)
# print(response)

In [8]:
def generate_response(system_input, user_input):

    # Format the input using the provided template
    prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"

    # Tokenize and encode the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=False)

    # Generate a response
    outputs = model.generate(inputs, max_length=1000, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    return response.split("### Assistant:\n")[-1]

In [9]:
# Set the padding token to the end-of-sequence token
# tokenizer.pad_token = tokenizer.eos_token

In [10]:
# Set the padding token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

def generate_response(system_input, user_input):
    # Format the input using the provided template
    prompt = f"### System:\n{system_input}\n### User:\n{user_input}\n### Assistant:\n"
    print(f"system_input: {system_input}")  # Print system_input to check its value
    
    # Tokenize and encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    print(f"inputs: {inputs}")
    # Generate a response
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=1000,
        num_return_sequences=1,
        # pad_token_id=tokenizer.eos_token_id  # Set pad_token_id to EOS token ID
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the assistant's response
    return response.split("### Assistant:\n")[-1]

In [11]:
system_input = """You are helping people with first aid. 
You will give them suggestions of how to fix their physical injury. 
If you cannot solve the problem, give them resources to look into. 
It is critical to limit your answers to the question and dont print anything else.
If you cannot answer the question, respond with 'Sorry, I dont know."""
user_input = "I have a papercut"
response = generate_response(system_input, user_input)
print(response)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


system_input: You are helping people with first aid. 
You will give them suggestions of how to fix their physical injury. 
If you cannot solve the problem, give them resources to look into. 
It is critical to limit your answers to the question and dont print anything else.
If you cannot answer the question, respond with 'Sorry, I dont know.
inputs: {'input_ids': tensor([[    1,   774,  2135, 28747,    13,  1976,   460,  8538,   905,   395,
           907, 11092, 28723, 28705,    13,  1976,   622,  2111,   706, 17278,
           302,   910,   298,  6293,   652,  5277, 11254, 28723, 28705,    13,
          3381,   368,  3573, 12049,   272,  2700, 28725,  2111,   706,  5823,
           298,   913,   778, 28723, 28705,    13,  1313,   349,  7276,   298,
          3607,   574, 11194,   298,   272,  2996,   304,  7286,  2682,  2424,
          1112, 28723,    13,  3381,   368,  3573,  4372,   272,  2996, 28725,
          9421,   395,   464, 23166, 28725,   315,  7286,   873, 28723,    13,
   

In [12]:
# examples = """Neutral: ""What to do if Cuts?", "How to cure Cuts?", "Which medicine to apply for Cuts?", "what to apply on cuts?", "Cuts""
# Aide: "Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied."

# Neutral: ""how do you treat abrasions?", "Do Abrasions cause scars?", "Abrasions", "what to do if abrasions?", "Which medicine to apply for abrasions?", "How to cure abrasions?""
# Aide: "Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.Cover a wound that bled with a thin layer of topical antibiotic ointment, like Bacitracin, or a sterile moisture barrier ointment, like Aquaphor. Cover it with a clean bandage or gauze. Gently clean the wound and change the ointment and bandage once per day.Watch the area for signs of infection, like pain or redness and swelling. See your doctor if you suspect infection."

# Neutral: ""How do you treat Sting?", "Stings", "What to do if you get a sting?", "Which medicine to apply if sting?""
# Aide: "Remove any stingers immediately. Some experts recommend scraping out the stinger with a credit card. Applying ice to the site may provide some mild relief. Apply ice for 20 minutes once every hour as needed. Wrap the ice in a towel or keep a cloth between the ice and skin to keep from freezing the skin. Taking an antihistamine such as diphenhydramine (Benadryl) or a nonsedating one such as loratadine (Claritin) will help with itching and swelling. Take acetaminophen (Tylenol) or ibuprofen (Motrin)for pain relief as needed. Wash the sting site with soap and water. Placing hydrocortisone cream on the sting can help relieve redness, itching, and swelling."

# Neutral: ""How to remove Splinters", "How to cure Splinters?", "What to do if I have splinters?", "How do you bring a splinter to the surface?""
# Aide: "1. SOAK IT IN EPSOM SALTS. Dissolve a cup of the salts into a warm bath and soak whatever part of the body has the splinter. Failing that, you can also put some of the salts onto a bandage pad and leave it covered for a day; this will eventually help bring the splinter to the surface. 2. VINEGAR OR OIL. Another simple way to draw out that stubborn splinter is to soak the affected area in oil (olive or corn) or white vinegar. Just pour some in a bowl and soak the area for around 20 to 30 minutes,"""

# messages = [
# {
# "role": "system",
# "content": """You are helping people with first aid. You will give them suggestions of how to fix their physical injury. 
# If you cannot solve the problem, give them resources to look into. 
# Make sure you are careful about the way you give your suggestions. 
# Be sure to be clear and concise about your answer. 
# It is critical to limit your answers to the question and dont print anything else.
# If you cannot answer the question, respond with 'Sorry, I dont know.""" + examples
# },
# {
# "role": "user",
# "content": 'Neutral: "I am going to meet my friend for a night out on the town."\nYoda:'
# }
# ]

# for model in ["Neural-Chat-7B","Hermes-2-Pro-Mistral-7B", "Yi-34B-Chat"]:
#     result = pg.Chat.create(
#         model,
#         messages=messages
#     )
#     print("="*71)
#     print(f"Using Model: {model}")
#     print(f"Neutral Text: I am going to meet my friend for a night out on the town.")
#     lines = result['choices'][0]['message']['content'].split('\n')
#     print(f"How would Yoda say this?: {lines[0]}")
#     print("="*71)

In [13]:
examples = """Neutral: ""What to do if Cuts?", "How to cure Cuts?", "Which medicine to apply for Cuts?", "what to apply on cuts?", "Cuts""
Aide: "Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied."

Neutral: ""how do you treat abrasions?", "Do Abrasions cause scars?", "Abrasions", "what to do if abrasions?", "Which medicine to apply for abrasions?", "How to cure abrasions?""
Aide: "Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and apply gentle pressure to the area to stop any bleeding.Cover a wound that bled with a thin layer of topical antibiotic ointment, like Bacitracin, or a sterile moisture barrier ointment, like Aquaphor. Cover it with a clean bandage or gauze. Gently clean the wound and change the ointment and bandage once per day.Watch the area for signs of infection, like pain or redness and swelling. See your doctor if you suspect infection."

Neutral: ""How do you treat Sting?", "Stings", "What to do if you get a sting?", "Which medicine to apply if sting?""
Aide: "Remove any stingers immediately. Some experts recommend scraping out the stinger with a credit card. Applying ice to the site may provide some mild relief. Apply ice for 20 minutes once every hour as needed. Wrap the ice in a towel or keep a cloth between the ice and skin to keep from freezing the skin. Taking an antihistamine such as diphenhydramine (Benadryl) or a nonsedating one such as loratadine (Claritin) will help with itching and swelling. Take acetaminophen (Tylenol) or ibuprofen (Motrin)for pain relief as needed. Wash the sting site with soap and water. Placing hydrocortisone cream on the sting can help relieve redness, itching, and swelling."

Neutral: ""How to remove Splinters", "How to cure Splinters?", "What to do if I have splinters?", "How do you bring a splinter to the surface?""
Aide: "1. SOAK IT IN EPSOM SALTS. Dissolve a cup of the salts into a warm bath and soak whatever part of the body has the splinter. Failing that, you can also put some of the salts onto a bandage pad and leave it covered for a day; this will eventually help bring the splinter to the surface. 2. VINEGAR OR OIL. Another simple way to draw out that stubborn splinter is to soak the affected area in oil (olive or corn) or white vinegar. Just pour some in a bowl and soak the area for around 20 to 30 minutes,"""

messages = [
{
"role": "system",
"content": """You are helping people with first aid. You will give them suggestions of how to fix their physical injury. 
If you cannot solve the problem, give them resources to look into. 
Make sure you are careful about the way you give your suggestions. 
Be sure to be clear and concise about your answer. 
It is critical to limit your answers to the question and dont print anything else.
If you cannot answer the question, respond with 'Sorry, I dont know.""" + examples
},
{
"role": "user",
"content": 'Neutral: "I broke my leg yesterday because I kicked someone on accident."\nAide:'
}
]

In [14]:
# Process each message
for message in messages:
  # Extract neutral text for the user
  neutral_text = message["content"].split("\n")[0].split(": ")[-1]

  # Generate Yoda-like text using the prompt function (use empty string for system_input)
  aide_text = generate_response(neutral_text, "")  # Pass empty string for system_input

  # Print results
  print("=" * 71)
  print(f"Neutral Text: {neutral_text}")
  print(f"How would Aide say this?: {aide_text}")
  print("=" * 71)

# # Process each message
# for message in messages:
#   # Extract neutral text for the user
#   neutral_text = message["content"].split("\n")[0].split(": ")[-1]

#   # Generate Yoda-like text using the prompt function
#   aide_text = generate_response(neutral_text, system_input="")

#   # Print results
#   print("=" * 71)
#   print(f"Neutral Text: {neutral_text}")
#   print(f"How would Aide say this?: {aide_text}")
#   print("=" * 71)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


system_input: You are helping people with first aid. You will give them suggestions of how to fix their physical injury. 
inputs: {'input_ids': tensor([[    1,   774,  2135, 28747,    13,  1976,   460,  8538,   905,   395,
           907, 11092, 28723,   995,   622,  2111,   706, 17278,   302,   910,
           298,  6293,   652,  5277, 11254, 28723, 28705,    13, 27332,  1247,
         28747,    13,    13, 27332, 21631, 28747,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Neutral Text: You are helping people with first aid. You will give them suggestions of how to fix their physical injury. 
How would Aide say this?:  For a minor cut or scrape, follow these steps:

1. Wash your hands with soap and water.
2. Gently clean the wound with warm water and mild soap.
3. Apply an antibiotic ointment to the cut or scrape.
4. Cover the area with a sterile bandage or gauze.
5. Change the bandage daily or whenever it gets wet or dirty.
6. Monitor the wound for signs of infection, such as increased redness, swelling, or pus.

For a more serious injury, seek medical attention immediately.
system_input: "I broke my leg yesterday because I kicked someone on accident."
inputs: {'input_ids': tensor([[    1,   774,  2135, 28747,    13, 28739, 28737,  8845,   586,  1942,
         12091,  1096,   315, 17444,  2493,   356,  8318,   611,    13, 27332,
          1247, 28747,    13,    13, 27332, 21631, 28747,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [15]:
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn

import intel_extension_for_pytorch as ipex   # to add intel GPU namespace (torch.xpu) to pytor|ch



In [16]:
if torch.xpu.is_available():
    def get_memory_usage():
        memory_reserved = round(torch.xpu.memory_reserved() / 1024**3, 3)
        memory_allocated = round(torch.xpu.memory_allocated() / 1024**3, 3)
        max_memory_reserved = round(torch.xpu.max_memory_reserved() / 1024**3, 3)
        max_memory_allocated = round(torch.xpu.max_memory_allocated() / 1024**3, 3)
        return memory_reserved, memory_allocated, max_memory_reserved, max_memory_allocated
   
    def print_memory_usage():
        device_name = torch.xpu.get_device_name()
        print(f"XPU available!! \n - Name: {device_name}")
        memory_reserved, memory_allocated, max_memory_reserved, max_memory_allocated = get_memory_usage()
        memory_usage_text = f" - XPU Memory: Reserved={memory_reserved} GB, Allocated={memory_allocated} GB, Max Reserved={max_memory_reserved} GB, Max Allocated={max_memory_allocated} GB"
        print(f"\r{memory_usage_text}", end="", flush=True)
    
    print_memory_usage()
    torch.xpu.empty_cache()

XPU available!! 
 - Name: Intel(R) Data Center GPU Max 1100
 - XPU Memory: Reserved=0.0 GB, Allocated=0.0 GB, Max Reserved=0.0 GB, Max Allocated=0.0 GB

In [17]:
from datasets import load_dataset

dataset = load_dataset("badri55/First_aid__dataset", split="train")

2024-04-20 14:54:08,475 - datasets - INFO - PyTorch version 2.0.1a0+cxx11.abi available.


In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
device = "xpu" #"xpu" if torch.xpu.is_available() else "cpu"
model_name = "microsoft/phi-1_5"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name).to(device) # move to xpu device if available, otherwise use cpu

In [19]:
from transformers import pipeline

question = dataset[40]["tag"]

print("Inference before fine-tuning:")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, truncation=True)
output = generator(question, max_length=42)
print("Question:", question)
# try:
#     print("Answer:", output[0]["generated_text"])
# except (KeyError, IndexError):
#     print("Error accessing generated text:", output)
print("Answer:", output[0]["generated_text"])


Inference before fine-tuning:


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Question: animal bite
Answer: animal bite.

4. What could John have done differently to avoid getting bitten by the snake?
Answer: John could have worn protective clothing, such as boots and long pants, and carried a snake


In [20]:
def format_dataset(sample):
    instruction = f"### Instruction\n{sample['patterns']}"
    context = f"### Context\n{sample['context_set']}" if len(sample["context_set"]) > 0 else None
    response = f"### Answer\n{sample['responses']}"
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    sample["text"] = f"{prompt}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(format_dataset)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)

# Split the dataset into train and test subsets
train_test_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

In [21]:
!pip install peft
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    r=16,
    lora_alpha=2,
    target_modules=["fc1", "fc2","Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
# 
model = get_peft_model(model, config)
model.print_trainable_parameters()
#model.gradient_checkpointing_enable()  # enable if low on VRAM

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
trainable params: 7,864,320 || all params: 1,426,135,040 || trainable%: 0.5514428703750243


In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        output_dir="output",
        bf16=True,
        use_ipex=True,
        max_grad_norm=0.6,
        weight_decay=0.01,
        group_by_length=True,
        optim="adamw_hf",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=30,
        max_steps=200,
        #num_train_epochs=3,
        report_to="wandb"
    )

In [23]:
!pip install wandb
from transformers import DataCollatorForLanguageModeling 
from transformers import Trainer
import os
os.environ["TOKENIZERS_PARALLELISM"] = "0"  # prevent warnings from training on process forking
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


max_steps is given, it will override any value given in num_train_epochs


In [24]:
if torch.xpu.is_available():
    torch.xpu.empty_cache()
results = trainer.train()


2024-04-20 14:54:23,881 - wandb.jupyter - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: knalabotu (ucsandiego). Use `wandb login --relogin` to force relogin


Step,Training Loss
30,2.139000
60,1.359600
90,1.049800
120,0.832600
150,0.758800
180,0.707200


In [27]:
trainer.save_model("./fine_tuned_model")


In [31]:
print("\nInference after fine-tuning with context:")
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model").to("xpu")
del fine_tuned_model
if torch.xpu.is_available():
    torch.xpu.empty_cache()
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model").to("xpu")
prompt = "what should i do if i pull a muscle?"
context = "Pulling a muscle often prevents people from walking and can get worse if you don't rest."
input_text = f"### Instruction\n{prompt}\n\n### Context\n{context}\n\n### Answer\n"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("xpu")

outputs = fine_tuned_model.generate(input_ids=input_ids, max_length=100, num_return_sequences=1, temperature=0.1, do_sample=True, num_beams=4)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Inference after fine-tuning with context:
### Instruction
what should i do if i pull a muscle?

### Context
Pulling a muscle often prevents people from walking and can get worse if you don't rest.

### Answer
If you pull a muscle, you may feel a sharp pain in the muscle. You may also feel a dull ache in the muscle.

### Examples
1. "How do you treat a pulled muscle?"
2. "Which medicine to take if I have a pulled muscle
